# Universidade de Fortaleza
## Mestrado em Informática Aplicada
### Ciência de Dados aplicada à Ciência da Cidade


# Schelling Model Analysis

This notebook explores the Schelling segregation model with different parameters to understand how individual preferences affect collective patterns of segregation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

## Model Implementation

In [ ]:
class SchellingModel:
    def __init__(self, grid_size, empty_fraction, group_proportions, tolerance):
        """
        Initialize the Schelling segregation model.
        
        Parameters:
        -----------
        grid_size : int
            Size of the square grid (grid_size x grid_size)
        empty_fraction : float
            Fraction of empty cells (0 to 1)
        group_proportions : list of float
            Proportions for each group (must sum to 1)
        tolerance : float
            Minimum fraction of similar neighbors required for satisfaction (0 to 1)
        """
        self.grid_size = grid_size
        self.empty_fraction = empty_fraction
        self.group_proportions = group_proportions
        self.tolerance = tolerance
        self.n_groups = len(group_proportions)
        
        # Initialize the grid
        # 0 = empty, 1, 2, ... = different groups
        self.grid = self._initialize_grid()
        
    def _initialize_grid(self):
        """Create initial random distribution of agents."""
        total_cells = self.grid_size ** 2
        n_empty = int(total_cells * self.empty_fraction)
        n_occupied = total_cells - n_empty
        
        # Create array with group assignments
        grid = np.zeros(total_cells, dtype=int)
        
        # Assign groups based on proportions
        current_idx = 0
        for group_id, proportion in enumerate(self.group_proportions, start=1):
            n_agents = int(n_occupied * proportion)
            grid[current_idx:current_idx + n_agents] = group_id
            current_idx += n_agents
        
        # Shuffle and reshape
        np.random.shuffle(grid)
        return grid.reshape(self.grid_size, self.grid_size)
    
    def _get_neighbors(self, x, y):
        """Get Moore neighborhood (8 neighbors) for cell at (x, y)."""
        neighbors = []
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                if dx == 0 and dy == 0:
                    continue
                nx, ny = (x + dx) % self.grid_size, (y + dy) % self.grid_size
                neighbors.append(self.grid[nx, ny])
        return neighbors
    
    def _is_satisfied(self, x, y):
        """Check if agent at (x, y) is satisfied with their neighborhood."""
        agent_type = self.grid[x, y]
        if agent_type == 0:  # Empty cell
            return True
        
        neighbors = self._get_neighbors(x, y)
        occupied_neighbors = [n for n in neighbors if n != 0]
        
        if len(occupied_neighbors) == 0:
            return True
        
        similar = sum(1 for n in occupied_neighbors if n == agent_type)
        similarity_ratio = similar / len(occupied_neighbors)
        
        return similarity_ratio >= self.tolerance
    
    def _get_empty_cells(self):
        """Return list of empty cell coordinates."""
        return list(zip(*np.where(self.grid == 0)))
    
    def step(self):
        """Perform one simulation step: move all unsatisfied agents."""
        unsatisfied = []
        
        # Find all unsatisfied agents
        for x in range(self.grid_size):
            for y in range(self.grid_size):
                if self.grid[x, y] != 0 and not self._is_satisfied(x, y):
                    unsatisfied.append((x, y))
        
        # Move unsatisfied agents to random empty cells
        empty_cells = self._get_empty_cells()
        
        for x, y in unsatisfied:
            if len(empty_cells) == 0:
                break
            
            # Choose random empty cell
            new_idx = np.random.randint(len(empty_cells))
            new_x, new_y = empty_cells[new_idx]
            
            # Move agent
            self.grid[new_x, new_y] = self.grid[x, y]
            self.grid[x, y] = 0
            
            # Update empty cells list
            empty_cells[new_idx] = (x, y)
        
        return len(unsatisfied)
    
    def run(self, max_steps=100):
        """Run simulation until convergence or max_steps."""
        history = [self.grid.copy()]
        unsatisfied_counts = []
        
        for step in range(max_steps):
            n_unsatisfied = self.step()
            history.append(self.grid.copy())
            unsatisfied_counts.append(n_unsatisfied)
            
            if n_unsatisfied == 0:
                print(f"Converged at step {step + 1}")
                break
        
        return history, unsatisfied_counts
    
    def get_satisfaction_rate(self):
        """Calculate fraction of satisfied agents."""
        satisfied = 0
        total = 0
        
        for x in range(self.grid_size):
            for y in range(self.grid_size):
                if self.grid[x, y] != 0:
                    total += 1
                    if self._is_satisfied(x, y):
                        satisfied += 1
        
        return satisfied / total if total > 0 else 0
    
    def visualize(self, title="Schelling Model"):
        """Visualize current state of the grid."""
        plt.figure(figsize=(8, 8))
        cmap = plt.colormaps.get_cmap('copper').resampled(self.n_groups + 1)
        plt.imshow(self.grid, cmap=cmap, vmin=0, vmax=self.n_groups)
        plt.title(f"{title}\nSatisfaction: {self.get_satisfaction_rate():.2%}")
        plt.colorbar(ticks=range(self.n_groups + 1), label='Group')
        plt.tight_layout()
        plt.show()

## Parameter Exploration

In [ ]:
# Example simulation with grid_size = 50
np.random.seed(42)

# Model parameters
grid_size = 100
empty_fraction = 0.1
group_proportions = [0.5, 0.5]  # Two equal groups
tolerance = 0.4  # 30% similar neighbors required

# Create and run model
model = SchellingModel(grid_size, empty_fraction, group_proportions, tolerance)

print("Initial state:")
model.visualize("Initial Configuration")

print(f"\nInitial satisfaction rate: {model.get_satisfaction_rate():.2%}")

# Run simulation
history, unsatisfied_counts = model.run(max_steps=100)

print(f"\nFinal satisfaction rate: {model.get_satisfaction_rate():.2%}")
model.visualize("Final Configuration")

# Plot convergence
plt.figure(figsize=(10, 5))
plt.plot(unsatisfied_counts, marker='o')
plt.xlabel('Step')
plt.ylabel('Number of Unsatisfied Agents')
plt.title('Convergence of Schelling Model')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()